In [12]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests, os
import h5py as h5
import glob
from os import walk
import time
from matplotlib.gridspec import GridSpec
from labexp import experiments,nano_lab
from mpl_toolkits.axes_grid1 import make_axes_locatable,ImageGrid, AxesGrid
from matplotlib.gridspec import GridSpec
import matplotlib as mpl
plt.style.use('/media/labfiles/lab-codes/puppy-project/src/plotstyle.mplstyle')


data =experiments('nano-lab','nsom','grating_InP',False)

data.data


[['exp_2022-04',
  'exp_2022-01',
  'exp_2022-09',
  'exp_2021-11',
  'exp_2022-05',
  'exp_2022-03'],
 ['exp_2022_19_04_nsom',
  'exp_2022_22_04_nsom',
  'exp_2022_23_04_nsom',
  'exp_2022_11_04_afm',
  'exp_2022_06_04_afm',
  'exp_2022_25_04_nsom',
  'exp_2022_18_04_nsom',
  'exp_2022_08_04_afm',
  'exp_2022_01_04_afm',
  'exp_2022_03_04_afm',
  'exp_2022_20_04_nsom',
  'exp_2022_07_04_afm',
  'exp_2022_24_04_nsom',
  'exp_2022_09_04_afm',
  'exp_2022_15_04_nsom',
  'exp_2022_10_04_afm',
  'exp_2022_05_04_afm',
  'exp_2022_02_04_afm'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['exp_2022_14_01_afm', 'exp_2022_19_01_afm', 'exp_2022_13_01_afm'],
 [],
 [],
 [],
 ['exp_2022_9_09_afm', 'exp_2022_8_09_afm'],
 [],
 [],
 ['exp_06-11_afm', 'exp_04-11_afm'],
 [],
 [],
 ['exp_2022_09_05_nsom',
  'exp_2022_11_05_nsom',
  'exp_2022_07_05_nsom',
  'exp_2022_06_05_nsom'],
 [],
 [],
 [],
 [],
 ['exp_2022_24_03_afm',
  'exp_2022_31_03_afm',
  'exp_2022